Digit Recognizer Using Convolution Neural Network
>   1. Import the Libraries
>   2. Load the data
>   3. Visualizing the shape of train and test data
>   4. Normalization of data
>   5. Visualization of image
>   6. Create the Convolution Model 
>        * Add the Convolution Layer
>        * Add the MaxPooling Layer
>        * Add the Flatten Layer 
>        * Add the Dense Layer
>   7. Compile the Model
>   8. Train the Model
>   9. Visualizing the Accuracy of Model
>   10. Prediction on Test data

1. Import the Libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from keras.callbacks import callbacks
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


2. Load the data

In [ ]:
df_train = pd.read_csv('../input/digit-recognizer/train.csv')
df_test = pd.read_csv('../input/digit-recognizer/test.csv')

3. Visualizing the shape of train and test data

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
height = 28
width = 28
classes =10

4. Normalization of data

In [ ]:
x_train = df_train.drop(['label'], axis = 1).to_numpy().reshape(42000, 28, 28, 1).astype('float32')
x_train /= 255.
y_train = tf.keras.utils.to_categorical(df_train['label'], classes)

x_test = df_test.to_numpy().reshape(28000, 28, 28, 1).astype('float32')
x_test /= 255.

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

5. Visualization of image

In [ ]:
g = plt.imshow(x_train[6,:,:,0])
print("Output = " +str(y_train[6,:]))

*Now, we will have to predict for test set. So that's why we will create the Convolution Model*

In [ ]:
g = plt.imshow(x_test[7,:,:,0])

6. Create the Convolution Model 
>    * Add the Convolution Layer
>    * Add the MaxPooling Layer

In [ ]:
model =  models.Sequential()
model.add(layers.Conv2D(filters = 32, kernel_size =(3, 3), activation = 'relu', input_shape = (28, 28, 1), padding = 'same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.3))
model.add(layers.Conv2D(filters = 64, kernel_size =(3, 3), activation = 'relu', input_shape = (28, 28, 1), padding = 'same'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.3))                            
model.add(layers.Conv2D(filters = 64, kernel_size =(3, 3), activation = 'relu', input_shape = (28, 28, 1), padding = 'same'))


In [ ]:
model.summary()

   * Add the Flatten Layer 
   * Add the Dense Layer

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation = 'softmax'))

In [ ]:
model.summary()

7. Compile the Model

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
  """Stop training when the loss is at its min, i.e. the loss stops decreasing.

  Arguments:
      patience: Number of epochs to wait after min has been hit. After this
      number of no improvement, training stops.
  """

  def __init__(self, patience=0):
    super(EarlyStoppingAtMinLoss, self).__init__()

    self.patience = patience

    # best_weights to store the weights at which the minimum loss occurs.
    self.best_weights = None

  def on_train_begin(self, logs=None):
    # The number of epoch it has waited when loss is no longer minimum.
    self.wait = 0
    # The epoch the training stops at.
    self.stopped_epoch = 0
    # Initialize the best as infinity.
    self.best = np.Inf

  def on_epoch_end(self, epoch, logs=None):
    current = logs.get('loss')
    if np.less(current, self.best):
      self.best = current
      self.wait = 0
      # Record the best weights if current results is better (less).
      self.best_weights = self.model.get_weights()
    else:
      self.wait += 1
      if self.wait >= self.patience:
        self.stopped_epoch = epoch
        self.model.stop_training = True
        print('Restoring model weights from the end of the best epoch.')
        self.model.set_weights(self.best_weights)

  def on_train_end(self, logs=None):
    if self.stopped_epoch > 0:
      print('Epoch %05d: early stopping' % (self.stopped_epoch + 1))

8. Train the Model

In [ ]:
from sklearn.model_selection import train_test_split
x_t, x_v, y_t, y_v = train_test_split(x_train, y_train, test_size=0.2,random_state = 42)

In [ ]:
x_t.shape

In [ ]:
x_v.shape

In [ ]:
y_t.shape

In [ ]:
y_v.shape

In [ ]:
history = model.fit(x_t, y_t,validation_data=(x_v, y_v), epochs=15,callbacks = [EarlyStoppingAtMinLoss()])

9. Visualizing the Accuracy of Model

*Now,  it's  time  to  see  a  prediction!*

In [ ]:
result = model.predict(x_v)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


y_class = np.argmax(result, axis = 1) 

y_check = np.argmax(y_v, axis = 1) 

cmatrix = confusion_matrix(y_check, y_class)

plt.figure(figsize=(15,8))
plt.title('Confusion matrix of the test/predicted digits ', weight='bold', fontsize=18)
sns.heatmap(cmatrix,annot=True,cmap="Reds",fmt="d",cbar=False)

In [ ]:
plt.plot(history.history['acc'], label='Train')
plt.plot(history.history['val_acc'], label = 'Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

10. Prediction on Test data

In [ ]:
y_predict = model.predict_classes(x_test, verbose=0)
m = plt.imshow(x_test[9,:,:,0])
print("Output = " +str(y_predict[9]))

In [ ]:
submissions=pd.DataFrame({"ImageId": list(range(1,len(y_predict)+1)),
                         "Label": y_predict})
submissions.to_csv("sub.csv", index=False, header=True)